In [1]:
import numpy as np
import pandas as pd
import re
import string
import pickle

In [2]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

In [3]:
with open('../static/model/model.pickle', 'rb') as f:
    model = pickle.load(f)

In [4]:
with open('../static/model/corpora/stopwords/english', 'r') as file:
    sw = file.read().splitlines()

In [5]:
vocab = pd.read_csv('../static/model/vocabulary.txt', header=None)
tokens = vocab[0].tolist()

In [6]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [7]:
def preprocessing(text):
    df = pd.DataFrame([text], columns=['tweet'])
    df["tweet"] = df["tweet"].apply(lambda x: " ".join(x.lower() for x in x.split()))
    df["tweet"] = df['tweet'].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE) for x in x.split()))
    df["tweet"] = df["tweet"].apply(remove_punctuations)
    df["tweet"] = df['tweet'].str.replace('\d+', '', regex=True)
    df["tweet"] = df["tweet"].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
    df["tweet"] = df["tweet"].apply(lambda x: " ".join(ps.stem(x) for x in x.split()))
    return df["tweet"]

In [8]:
def vectorizer(ds, vocabulary):
    vectorized_lst = []
    
    for sentence in ds:
        sentence_lst = np.zeros(len(vocabulary))
        
        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_lst[i] = 1
                
        vectorized_lst.append(sentence_lst)
        
    vectorized_lst_new = np.asarray(vectorized_lst, dtype=np.float32)
    
    return vectorized_lst_new

In [9]:
def get_prediction(vectorized_text):
    prediction = model.predict(vectorized_text)
    if prediction == 1:
        return 'negative'
    else:
        return 'positive'

In [10]:
txt = "bad product"
preprocessed_txt = preprocessing(txt)
vectorized_txt = vectorizer(preprocessed_txt, tokens)
prediction = get_prediction(vectorized_txt)
prediction

'negative'